In [1]:
# Importing the required libraries
import os
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                          AutoConfig, Trainer, TrainingArguments, DataCollatorForTokenClassification)

# Importing custom classes for loading and labeling the corpus
from utils.corpusprocessor import CorpusType
from utils.corpusprocessor import CorpusLoader
from utils.labeler import Labeler

2023-10-15 18:12:37.150453: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-15 18:12:37.150482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Loading the corpus using the custom CorpusLoader class
corpus = CorpusLoader()
labeler = Labeler( tags=(1, 2),
                 regexes=(r'[^\S\r\n\v\f]', r'\u200c'),
                 chars=(" ", "‌"),
                 class_count=2,
                 )



In [4]:
data = corpus.load_bijan(CorpusType.whole_raw)
labeler.set_text(data, corpus_type=CorpusType.whole_raw)
chars, labels = labeler.labeler()
chars = chars[:3000]
labels = labels[:3000]

In [5]:
print(chars[:30])
print(labels[:30])
print('Character Length:',len(chars))
print(len(labels))

['#', 'ا', 'و', 'ل', 'ي', 'ن', 'س', 'ي', 'ا', 'ر', 'ه', 'خ', 'ا', 'ر', 'ج', 'ا', 'ز', 'م', 'ن', 'ظ', 'و', 'م', 'ه', 'ش', 'م', 'س', 'ي', 'د', 'ي', 'د']
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Character Length: 3000
3000


In [6]:
model_dir = "./Model2111/"
# pretrained_model = "HooshvareLab/bert-base-parsbert-uncased"
pretrained_model = "bert-base-multilingual-uncased"


tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = AutoConfig.from_pretrained(pretrained_model)
config.num_labels = 3

model = AutoModelForTokenClassification.from_config(config)

In [13]:
chars = [tokenizer.encode(char)[1] for char in chars]a
chars.insert(0, 2)  # Adding CLS token at the beginning of each sequence "2 is the tokenized "[CLS]" "
chars.append(4)  # Adding SEP token at the end of each sequence "4 is the tokenized "[SEP]" "
labels.insert(0,0) # # Adding CLS token label at the beginning of each sequence
labels.append(0) 

In [8]:
data = [{"text": " ".join(chars), "labels": labels}]
del labels,chars


NameError: name 'tokenizer' is not defined

In [18]:



        
# Creating a Dataset object from the tokenized inputs and labels
dataset = Dataset.from_dict({"input_ids": chars, "labels": labels})

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Setting up training arguments for the Trainer class
training_args = TrainingArguments(
    output_dir=model_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="no",
    save_strategy="no"
)

# Initializing a Trainer instance with the model, training arguments, dataset, tokenizer, and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator
)



In [21]:
# Training the model on the dataset
trainer.train()


trainer.save_model(model_dir + "model/") 

IndexError: tuple index out of range

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = AutoConfig.from_pretrained(pretrained_model)
config.num_labels = 3

model = AutoModelForTokenClassification.from_config(config)


In [8]:
model


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e